In [ ]:
import os                          # for operating system work
from dotenv import load_dotenv     # to load environment variables 
from langchain_groq import ChatGroq # for the chat module of the Groq

load_dotenv() # loading all the environment variables from the .env file
groq_api_key = os.getenv("GROQ_API_KEY") # get the groq api key

# Create Model from Groq, using the API Key and Model name
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I am Dinesh, I work at Accenture as an Associate Director")])

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, I am Dinesh, I work at Accenture as an Associate Director"),
        AIMessage(content="Hi Dinesh, it's nice to meet you! \n\nThat's great you're an Associate Director at Accenture. What kind of work do you do there?  \n\nI'm eager to learn more about your role and experience.\n"),
        HumanMessage(content="Hey whats my name and what do I do")
    ]

)

AIMessage(content="You are Dinesh, and you are an Associate Director at Accenture.  \n\nIs there anything else you'd like to tell me about your work there? 😊  I'm here to listen! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.00266732, 'queue_time': 0.070719377, 'total_time': 0.086303684}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--486ef362-c0c5-4569-b8e0-3721d312c576-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
### Message History
#This imports the ChatMessageHistory class, which is a 
# concrete implementation of BaseChatMessageHistory. It's used to store chat messages.
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
# This is a crucial class that wraps a runnable (like our ChatGroq model) and 
# imbues it with the ability to manage conversation history.
from langchain_core.runnables.history import RunnableWithMessageHistory

#This line initializes an empty dictionary named store
store={}
#This defines a function named get_session_history that takes a sessionid (a string) as 
# input and is type-hinted to return an object conforming to BaseChatMessageHistory.
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#This is where the magic happens for incorporating message history.
#RunnableWithMessageHistory is instantiated with two arguments:
# model: This is the ChatGroq model instance we created earlier. 
# This is the "runnable" that will be enhanced with history capabilities.
# later when the invoke method is called on with_message_history, it will use the 
# stored model and get_session_history to call get_session_history

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
#This dictionary defines the configuration for the invocation.
# "configurable": This key is used to pass configuration options to 
# runnables that support them.

config1={"configurable":{"session_id":"chat1"}}

In [8]:
#This line invokes the with_message_history runnable.
response = with_message_history.invoke(
    [HumanMessage(content="My name is dinesh and I am an associate director at Accenture")],
    config=config1 #This passes the config dictionary we defined, which includes the session_id
)

In [12]:
response.content

"Hello Dinesh! It's nice to meet you. Being an associate director at Accenture is quite impressive. \n\nIs there anything I can help you with today? Perhaps you have a question about Accenture, need help brainstorming ideas, or just want to chat?  I'm here to assist in any way I can. \n\n"

In [9]:
## change the config, it should not remember
config2={"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config2 #This passes the config dictionary we defined, which includes the session_id
)


AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52, 'completion_time': 0.069090909, 'prompt_time': 0.001292109, 'queue_time': 0.070070933, 'total_time': 0.070383018}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--beea92e7-15b2-4bcd-a1a8-5d06ef2595fb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

### Runnable protocol
ChatPromptTemplate is a core component of LangChain and is a full-fledged Runnable.

This means it adheres to the Runnable protocol and has methods like .invoke(), .batch(), .stream(), and .transform(), just like other runnables you've seen (e.g., ChatGroq, RunnablePassthrough).

### Using as Runnable 
We have already been using ChatPromptTemplate as a runnable in the code below. The pipe (|) operator from LangChain Expression Language (LCEL) only works between Runnable objects.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
#ChatPromptTemplate.from_messages is a class method (a method called directly on the class, 
# not an instance) that allows you to construct a ChatPromptTemplate from a list of "message templates.
# " Each element in the list represents a part of the conversation, specifying who is speaking (role)
#  and what their message is (content).It's designed specifically for chat models (like your ChatGroq model)
#  because chat models understand conversational turns (system messages, human messages, AI messages, etc.)
prompt=ChatPromptTemplate.from_messages (
    [
        ("system","You are a help assitant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
        # MessagesPlaceholder(variable_name="messages") is a crucial component in the ChatPromptTemplate. 
        # Its primary purpose is to act as a placeholder for the entire chat history. 
        # Its basically the name of the variable which you can assign new messages
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="My name is Pawan Deep Singh and I am the friend of Dinesh living in Bhopal India")]})

AIMessage(content="Hello Pawan Deep Singh, it's nice to meet you! \n\nI understand you're a friend of Dinesh who lives in Bhopal, India. How can I help you today? \n\nDo you have any questions about Bhopal, India? Or perhaps you'd like me to help you find something specific?  \n\nI'm here to assist you in any way I can. 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 46, 'total_tokens': 133, 'completion_time': 0.158181818, 'prompt_time': 0.001630099, 'queue_time': 0.069961276, 'total_time': 0.159811917}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7b557c92-275d-4dc8-a655-0388255001be-0', usage_metadata={'input_tokens': 46, 'output_tokens': 87, 'total_tokens': 133})

In [ ]:
## Create a new config called chat3
config3={"configurable":{"session_id":"chat3"}}
# We passed chain instead of the model here.
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

response = with_message_history.invoke(
    [HumanMessage(content="My name is Dinesh")],
    config=config3 #This passes the config dictionary we defined, which includes the session_id
)

response.content


"Hello Dinesh! \n\nIt's nice to meet you.  I'm ready to help with any questions you have. \n\nWhat can I do for you today? 😊  \n"

In [11]:
# Add more complexity, change the prompt to take a variable language 
prompt=ChatPromptTemplate.from_messages (
    [
        ("system","You are a help assitant. Answer all the questions to the best of your abaility in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
# I have to rerun the prompt here because the defination is stored and its not a link
chain=prompt|model

In [19]:

#ChatPromptTemplate is designed to take a dictionary as its input if it contains multiple variables.
# It looks for keys in this input dictionary that match the placeholders

response = chain.invoke(
    {"messages":[HumanMessage(content="My name is Dinesh")],
     "language":"hindi"
     }
)
response.content

'नमस्ते दिनेश! \n\nमैं आपकी सहायता करने के लिए यहाँ हूँ। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी क्षमताओं के अनुसार उत्तर देने की पूरी कोशिश करूँगा। \n\nआपको क्या जानना है? 😊 \n'

In [12]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
    )


In [13]:
## Create a new config called chat4
config4={"configurable":{"session_id":"chat4"}}
response4=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Dinesh")],
     "language":"Hindi"},
    config=config4
)
response4.content

'नमस्ते Dinesh! \n\nमुझे आपकी मदद करने में खुशी हो रही है। आप क्या जानना चाहते हैं? \n\nक्या कोई विशेष प्रश्न है या मैं कुछ भी और कैसे मदद कर सकता हूँ? \n'

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
#trim_messages: This is a crucial new component. It's a runnable factory 
# (meaning, it's a function that returns a Runnable instance) designed to intelligently shorten
#  a list of chat messages based on a token limit. This is vital for managing LLM context 
# windows and controlling costs

#trim_messages: This is a crucial new component. It's a runnable factory (meaning, 
# it's a function that returns a Runnable instance) designed to intelligently shorten a list of chat 
# messages based on a token limit. This is vital for managing LLM context windows and controlling costs
trimmer=trim_messages(
    max_tokens = 60, #This sets the maximum number of tokens the trimmed message history should occupy. 
    strategy = "last", #This dictates how messages are removed if max_tokens is exceeded.
    token_counter=model, #This is very important. trim_messages needs a way to count tokens accurately. 
                         #By passing your model (the ChatGroq instance), you're providing a 
                         # token counter that understands how Groq counts tokens, ensuring the max_tokens 
                         # limit is respected precisely for your chosen LLM.
    include_system=True, #the system message (if present) will always be included
    allow_partial=False, #If True, it might allow a message to be partially included 
                         #if splitting it fits the token limit.
    start_on="human" #means it will ensure that the first message
                     # kept in the history, after system messages, is a HumanMessag

)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Diensh"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Kheer Mohan Bengali Sweet"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

#initially max_tokens were set to 70 and it returned all value, when changed to 45 it trimmed messages


In [16]:
# itemgetter: This is a utility function from Python's operator module.
# It's often used in LangChain's LCEL for extracting specific items (keys) from an input dictionary.
    # In simple language it extracts the messages you pass as a list
from operator import itemgetter
#This is another powerful LCEL component. It literally "passes through" its input as its output.
# It's often used with .assign() to inject or modify keys in the input dictionary without changing 
# the original input.
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
# The defination of chain is given above, but actual invocation happens after below code is invoked.
# messages present in the dictionary is passed to the itemgetter which converts it to the list
# The list is then passed to the trimmer.
response=chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Which Bengali Sweet, I like?")],
        "language":"English"
    }
)

response.content

"You said you like Kheer Mohan!  🍰 \n\nIs there anything else you'd like to talk about?  Maybe other delicious Bengali sweets?  😋\n"